The KNN algorithm is very simple to implement, as it does not need to be trained. The training phase merely stores the training data. For each test point, we calculate the distance of that data point to every existing data point and find the $K$ closest ones. What we return is the the most common amongst the top k classification nearest to the test point. Here's the pseudocode for _K_ Nearest Neighbors:

```
kNN:

    Learn:
        Store training set T to X_train: X_train <-- T


    Predict:
        for every point xp in X_predict:
            for every point x in X_train:
                calculate the distance d in D between x and xp
            sort D in increasing order
            take the "k" items in X_train with the smallest distances to x
            return the majority class among these k items
```

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and evey other point in the data!


# Distance Metrics - Note

As explained above, KNN assigns a class to the test point based on the majority class of  KK  nearest neighbours. In general, euclidean distance is used to find nearest neighbours, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbours. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is Mahalanobis distance. Here's the wikipedia page on Mahalanobis distance. https://en.wikipedia.org/wiki/Mahalanobis_distance Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit learn scaling package. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [162]:
import pandas as pd
from statistics import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [1]:
def distance(vec1, vec2, c=2, verbose=False):
    if not len(vec1) == len(vec2):
        print('vec1 and vec2 need to be of same dimension')
        return 0
    if verbose == True:
        dm = 'Manhattan' if c == 1 else 'Euclidean' if c == 2 else 'Minkowski'
        print('{} distance between vec1 and vec2:'.format(dm))
    return np.power(sum([np.abs(vec1[i] - vec2[i])**c for i in range(len(vec1))]), 1/c)

In [139]:
class kNN():
    '''
    
    '''
    def __init__(self):
        self.X_train = []
        self.y_train = []
        self.k = 1

    # Learn:
    # Store training set T to X_train: X_train <-- T
    def fit(self, X_train, y_train, k=1):
        self.X_train = X_train
        self.y_train = y_train
        self.k = k
        
    def predict(self, X_test, regr=False):
        y_pred = []
        
        # for every point xp in X_predict:
        for el_p in X_test:
            dst_list = []

            # for every point x in X_train:
            for idx, el_t in enumerate(self.X_train):
                
                # calculate the distance d in D between el_p and el_t
                dist = distance(el_p, el_t, c=2)
                dst_list.append((dist, idx))

            # sort dst_list in increasing order 
            k_dist = sorted(dst_list)[:self.k]
            
            # take the "k" items in y_train with the smallest distances to x
            k_dist2 = [self.y_train[x[1]] for x in k_dist]
            
            # return the majority class among these k items
            if regr == False:
                y_pred.append(mode(k_dist2))
            if regr == True:
                y_pred.append(sum(k_dist2)/len(k_dist2))
        return y_pred

In [152]:
# Import iris dataset
df = pd.read_csv('iris.csv')

In [154]:
# Create a copy
df_copy = df.copy()

In [159]:
# Encode the species str label as integer using sklearn LabelEncoder
le = LabelEncoder()
le.fit(df['species'].unique())
df['species'] = le.transform(df['species'].values)

In [161]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [173]:
# 
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [201]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

In [202]:
knn3 = kNN()
knn3.fit(X_train.values, y_train.values, k=5)

In [203]:
from sklearn.metrics import accuracy_score, precision_score, classification_report

In [204]:
y_pred = knn3.predict(X_test.values)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.94      1.00      0.97        15
           2       1.00      0.92      0.96        12

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

